In [1]:
import pandas as pd
import urllib2
import time
import json

    

In [2]:
studentBehavior = pd.read_json("studentBehaviorInfoOver40Class_1213.json", orient="index")

In [3]:
print studentBehavior[0]
# print studentBehavior[0]["memberId"]
# print (studentBehavior[0]["chosenVideo"])
print (studentBehavior[0]["listenScore"])


## Desired json structure
# [
#     {
#         "memberId": 12345,
#         "videoSequence":
#             [
#                 3980,
#                 7680,
#                 1235
#             ],
#         "skippingRatioSequence":
#             [
#                 0,
#                 0,
#                 0.3
#             ],
#         "scoreSequence":
#             [
#                 80,
#                 70,
#                 75
#             ],
#         "dictionarySequence":
#             [
#                 0,
#                 0,
#                 0
#             ],
#         "skippingIndex": 10,
#         "clusterSequence":
#             [
#                 "C1",
#                 "C1",
#                 "C2"
#             ]
        
#     }
# ]



chosenVideo        [3913, 5186, 4974, 4802, 3711, 5881, 7126, 5797]
listenScore       [{u'postId': 3913, u'score': 93}, {u'postId': ...
memberId                                                      50679
vocabularyList    [{u'postId': u'5797', u'word': u'devil'}, {u'p...
Name: 0, dtype: object
[{u'postId': 3913, u'score': 93}, {u'postId': 3913, u'score': 91}, {u'postId': 3913, u'score': 92}, {u'postId': 3913, u'score': 92}, {u'postId': 3913, u'score': 92}, {u'postId': 5186, u'score': 93}, {u'postId': 5186, u'score': 94}, {u'postId': 5186, u'score': 93}, {u'postId': 4974, u'score': 90}, {u'postId': 4974, u'score': 80}, {u'postId': 4974, u'score': 77}, {u'postId': 4974, u'score': 96}, {u'postId': 4974, u'score': 86}, {u'postId': 4974, u'score': 97}, {u'postId': 4974, u'score': 90}, {u'postId': 4974, u'score': 95}, {u'postId': 4974, u'score': 95}, {u'postId': 4974, u'score': 98}, {u'postId': 4974, u'score': 91}, {u'postId': 4974, u'score': 86}, {u'postId': 4974, u'score': 95}, {u'postId':

In [10]:
lenStudentBehavior = len(pd.read_json("studentBehaviorInfoOver40Class_1213.json"))
usersProfilesJsonWithIndexes = {}
for index in range(lenStudentBehavior):
    if studentBehavior[index]["memberId"] not in usersProfilesJsonWithIndexes:
        studentProfile                          = {}
        studentProfile["memberId"]              = studentBehavior[index]["memberId"]
        studentProfile["videoSequence"]         = []
        studentProfile["skippingRatioSequence"] = []
        studentProfile["scoreSequence"]         = []
        studentProfile["dictionarySequence"]    = []
        studentProfile["skippingIndex"]         = -1
        studentProfile["clusterSequence"]       = []
        usersProfilesJsonWithIndexes[studentBehavior[index]["memberId"]] = studentProfile
    
    usersProfilesJsonWithIndexes[studentBehavior[index]["memberId"]]["videoSequence"].extend(studentBehavior[index]["chosenVideo"])

fileStudentProfile = open("studentProfile.json", "w")
fileStudentProfile.write("[")
lenObjects = len(usersProfilesJsonWithIndexes)
it = 0
for key, value in usersProfilesJsonWithIndexes.items():
    json.dump(value,fileStudentProfile)
    if it != (lenObjects-1):
        fileStudentProfile.write(", ")
    it = it + 1
    


fileStudentProfile.write("]")
fileStudentProfile.close()